In [1]:
# computable_analysis_demo.py
# Lightweight computable analysis framework (Cauchy representations + constructive algorithms)
# - Real: Cauchy representation via approx(k) -> Fraction within 2^{-k}
# - Basic arithmetic (add, sub, mul, neg)
# - exp_real (Taylor-series constructive approximation)
# - bisection_root (guaranteed root in sign-change interval)
# - integrate_adaptive (Simpson-style adaptive integration)
#
# Demo at the bottom computes:
#  - exp(1) to 30 bits,
#  - sqrt(2) via bisection to 40 bits,
#  - integral of exp(x) from 0 to 1 to 30 bits.

from fractions import Fraction
from math import factorial
import math

# Utility: 2^{-k} as Fraction
def two_pow_neg(k: int) -> Fraction:
    return Fraction(1, 1 << k)

class Real:
    """
    Represent a real number by a function approx(k) returning a Fraction q such that |q - x| <= 2^{-k}.
    The class stores an approximation function and provides arithmetic/composition that produce new Real objects.
    """
    def __init__(self, approx_func):
        self._approx = approx_func

    def approx(self, k: int) -> Fraction:
        if k < 0:
            return self._approx(0)
        return self._approx(k)

    @staticmethod
    def from_rational(q: Fraction):
        return Real(lambda k: q)

    @staticmethod
    def from_float(f: float, max_prec=60):
        q = Fraction(f).limit_denominator(10**12)
        return Real.from_rational(q)

    @staticmethod
    def from_cauchy(approx_func):
        return Real(approx_func)

    # Basic arithmetic with conservative precision propagation
    def add(self, other: 'Real') -> 'Real':
        def approx_sum(k):
            # Ask for k+2 precision from operands (safety margin)
            a = self.approx(k+2)
            b = other.approx(k+2)
            return a + b
        return Real(approx_sum)

    def neg(self) -> 'Real':
        return Real(lambda k: -self.approx(k))

    def sub(self, other: 'Real') -> 'Real':
        return self.add(other.neg())

    def mul(self, other: 'Real') -> 'Real':
        def approx_mul(k):
            # Conservative choice: ask for higher precision m = k+6
            m = k + 6
            a = self.approx(m)
            b = other.approx(m)
            return a * b
        return Real(approx_mul)

    def scale_rational(self, r: Fraction) -> 'Real':
        return Real(lambda k: r * self.approx(k))

# Constructors
def exact(q: int) -> Real:
    return Real.from_rational(Fraction(q))

def rational(p: int, q: int) -> Real:
    return Real.from_rational(Fraction(p, q))

# Exponential function: construct Real representing exp(x)
def exp_real(x: Real) -> Real:
    """
    Construct a Real representing exp(x).
    For requested precision 2^{-k}:
      - request an approximation xr of x with precision m = k + safety
      - compute Taylor series of exp at xr
      - truncate when the next term <= threshold (conservative)
    This returns a rational S approximating exp(x) (with conservative margins).
    """
    def approx_exp(k: int) -> Fraction:
        safety = 6
        m = k + safety
        xr = x.approx(m)  # Fraction within 2^{-m} of x
        # Work with rational xr. Build Taylor series sum_{j=0..N} xr^j / j!
        target_term = Fraction(1, 1 << (k + 4))
        term = Fraction(1, 1)  # xr^0 / 0!
        S = Fraction(0, 1)
        j = 0
        # Loop until next term magnitude <= target_term
        while True:
            S += term
            j += 1
            term = term * xr / j
            if abs(term) <= target_term:
                break
            if j > 5000:
                raise RuntimeError("Series did not converge fast enough (xr too large?)")
        # S approximates exp(xr); we used margins to account for xr approximating x
        return S
    return Real(approx_exp)

# Bisection root finder for sign-changing continuous functions f: [a,b] -> R.
def bisection_root(f_callable, a: Fraction, b: Fraction, k: int):
    """
    f_callable(x, m) -> Fraction approximating f(x) within 2^{-m}.
    a, b: Fraction endpoints with f(a)*f(b) < 0 (sign change)
    k: target bits: return a Fraction r with |r - root| <= 2^{-k}
    """
    left = a
    right = b
    max_iter = k + 80  # generous safety cap

    # initial sign at left with some precision
    prec0 = 40
    fa = f_callable(left, prec0)
    fb = f_callable(right, prec0)
    if fa == 0:
        return left
    if fb == 0:
        return right
    if fa * fb > 0:
        raise ValueError("f(a) and f(b) must have opposite signs (sign change required).")

    for i in range(max_iter):
        mid = Fraction(left + right, 2)
        prec = max(40, k + 20)  # precision for function evaluations
        fm = f_callable(mid, prec)
        if abs(fm) <= two_pow_neg(prec):
            # mid is zero within evaluation precision: accept
            return mid
        # decide which subinterval contains sign change
        if fa * fm <= 0:
            right = mid
            fb = fm
        else:
            left = mid
            fa = fm
        if right - left <= two_pow_neg(k):
            return Fraction(left + right, 2)
    raise RuntimeError("Bisection did not converge within iteration bound")

# Adaptive Simpson-like integration with sampling and Simpson error estimate
def integrate_adaptive(f_callable, a: Fraction, b: Fraction, k: int):
    """
    f_callable(x, m) -> Fraction approximating f(x) within 2^{-m}.
    Returns Fraction approximating integral_{a}^{b} f(x) dx within 2^{-k}.
    Uses composite Simpson with doubling N until estimated error <= 2^{-k}.
    """
    def sample_partition(N):
        if N % 2 != 0:
            raise ValueError("N must be even for Simpson's rule.")
        h = Fraction(b - a, N)
        prec = k + 12
        fvals = []
        for i in range(N+1):
            x = a + h * i
            fvals.append(Fraction(f_callable(x, prec)))
        s = fvals[0] + fvals[-1]
        for i in range(1, N):
            if i % 2 == 1:
                s += 4 * fvals[i]
            else:
                s += 2 * fvals[i]
        return (h * s) / 3

    N = 4
    target = two_pow_neg(k)
    prev = sample_partition(N)
    for _ in range(25):
        N *= 2
        curr = sample_partition(N)
        est_err = abs(curr - prev) / 15  # Simpson error estimator
        prev = curr
        if est_err <= target:
            return curr
    raise RuntimeError("Adaptive integration did not reach required precision")

# -----------------------
# Demo / tests
# -----------------------
if __name__ == "__main__":
    # Exact rationals and zero/one
    one = exact(1)
    zero = exact(0)

    # Example: exp(1)
    exp1 = exp_real(one)
    prec_bits = 30
    approx_exp1 = exp1.approx(prec_bits)  # Fraction
    print("Approx exp(1) (30 bits) as Fraction:", approx_exp1)
    print("Approx exp(1) as float:", float(approx_exp1))
    print("Reference math.e:", math.e)
    print("Error (float):", abs(float(approx_exp1) - math.e))

    # Example: root of x^2 - 2 in [1,2]:
    def f_sqrt2_callable(x_frac: Fraction, m: int):
        # compute x^2 - 2 exactly using rationals (no Real required here)
        return x_frac * x_frac - Fraction(2,1)

    root_approx = bisection_root(f_sqrt2_callable, Fraction(1,1), Fraction(2,1), 40)  # 40 bits precision
    print("\nApprox root of x^2-2 (40 bits):", root_approx)
    print("Root as float:", float(root_approx))
    print("Reference sqrt(2):", math.sqrt(2))
    print("Error (float):", abs(float(root_approx) - math.sqrt(2)))

    # Example: integral of exp(x) from 0 to 1
    def f_exp_callable(x_frac: Fraction, m: int):
        # compute exp(x_frac) by creating a Real from the rational point and using exp_real
        rx = Real.from_rational(x_frac)
        ex = exp_real(rx)
        return ex.approx(m+6)

    integral_exp_0_1 = integrate_adaptive(f_exp_callable, Fraction(0,1), Fraction(1,1), 30)
    print("\nIntegral of exp(x) from 0 to 1 (approx):", integral_exp_0_1)
    print("Integral as float:", float(integral_exp_0_1))
    print("Reference e-1:", math.e - 1)
    print("Error (float):", abs(float(integral_exp_0_1) - (math.e - 1)))


Approx exp(1) (30 bits) as Fraction: 8463398743/3113510400
Approx exp(1) as float: 2.718281828446759
Reference math.e: 2.718281828459045
Error (float): 1.2286172079711832e-11

Approx root of x^2-2 (40 bits): 3109888511975/2199023255552
Root as float: 1.414213562372879
Reference sqrt(2): 1.4142135623730951
Error (float): 2.1604940059205546e-13

Integral of exp(x) from 0 to 1 (approx): 78126211255128015624347790907007098797435541/45467635131381134766479166669905417207808000
Integral as float: 1.718281829028015
Reference e-1: 1.718281828459045
Error (float): 5.689699822397642e-10
